In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
from langchain.embeddings.openai import OpenAIEmbeddings

import chromadb
from chromadb.config import Settings

from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

True

In [2]:
pdf_folder = "/Users/prudvikamtam/Documents/Projects/LLM/JetBlue_LLM/documents"

pdf_files = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.endswith('.pdf')]
all_documents = []

# Load documents from each PDF file
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    documents = loader.load()
    all_documents.extend(documents)

# Split documents into smaller chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(all_documents)

print(texts)


Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 25 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 29 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 41 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 23 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 51 0 (offset 0)
Ignoring wrong pointing object 70 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 35 0 (offset 0)
Ignoring wrong pointing object 59 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 72 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wron

[Document(metadata={'source': '/Users/prudvikamtam/Documents/Projects/LLM/JetBlue_LLM/documents/Travel Certificates | JetBlue.pdf', 'page': 0}, page_content='7/28/24, 23:19Travel Certificates | JetBlue\nPage 1 of 9https://www.jetblue.com/help/travel-certificatesHow to Redeem TravelCertificatesTravel certificates provide confirmed travel with JetBlue. Help\n●Terms And Conditions For Travel Certificates●Information Needed For New Booking Requests●Blackout Dates●Have Questions About Travel Certificates?'), Document(metadata={'source': '/Users/prudvikamtam/Documents/Projects/LLM/JetBlue_LLM/documents/Travel Certificates | JetBlue.pdf', 'page': 1}, page_content='7/28/24, 23:19Travel Certificates | JetBlue\nPage 2 of 9https://www.jetblue.com/help/travel-certificatesTerms and conditions for travelcertificatesCertificate travel is always subject to availability and blackout dates. Change/cancelfees apply per person.\u202fTravel must be booked and completed prior to the expirationdate listed on

In [3]:
len(texts)

318

In [4]:
texts[0]

Document(metadata={'source': '/Users/prudvikamtam/Documents/Projects/LLM/JetBlue_LLM/documents/Travel Certificates | JetBlue.pdf', 'page': 0}, page_content='7/28/24, 23:19Travel Certificates | JetBlue\nPage 1 of 9https://www.jetblue.com/help/travel-certificatesHow to Redeem TravelCertificatesTravel certificates provide confirmed travel with JetBlue. Help\n●Terms And Conditions For Travel Certificates●Information Needed For New Booking Requests●Blackout Dates●Have Questions About Travel Certificates?')

In [5]:
dict(texts[0])

{'id': None,
 'metadata': {'source': '/Users/prudvikamtam/Documents/Projects/LLM/JetBlue_LLM/documents/Travel Certificates | JetBlue.pdf',
  'page': 0},
 'page_content': '7/28/24, 23:19Travel Certificates | JetBlue\nPage 1 of 9https://www.jetblue.com/help/travel-certificatesHow to Redeem TravelCertificatesTravel certificates provide confirmed travel with JetBlue. Help\n●Terms And Conditions For Travel Certificates●Information Needed For New Booking Requests●Blackout Dates●Have Questions About Travel Certificates?',
 'type': 'Document'}

In [6]:
metadatas = [item.metadata for item in texts]
contents = [item.page_content for item in texts]
ids = [str(i) for i in range(len(metadatas))]

In [10]:
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

openai_ef = OpenAIEmbeddingFunction(
        api_key=os.environ.get('OPENAI_API_KEY'),
        model_name="text-embedding-ada-002"
        )

ST_EF = SentenceTransformerEmbeddingFunction()

/Users/prudvikamtam/Documents/Projects/LLM/env/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [11]:
# embeddings = openai_ef(contents)
embeddings = ST_EF(contents)

In [12]:
client = chromadb.PersistentClient(path="/Users/prudvikamtam/Documents/Projects/LLM/JetBlue_LLM/chromadb2")

collection = client.get_or_create_collection(name="JetBlueHelp_stef") 

In [13]:
collection.add(
    documents=contents,
    embeddings=embeddings,
    metadatas=metadatas,
    ids=ids
)


In [14]:
metadatas[0]

{'source': '/Users/prudvikamtam/Documents/Projects/LLM/JetBlue_LLM/documents/Travel Certificates | JetBlue.pdf',
 'page': 0}